In [76]:
!pip install transformers
#!pip install huggingface_hub
#!pip install datasets
!pip install torch

In [10]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [11]:
datasets = load_dataset("piqa")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3084 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1838 [00:00<?, ? examples/s]

Dataset piqa downloaded and prepared to C:/Users/shwet/.cache/huggingface/datasets/piqa/plain_text/1.1.0/6c611c1a9bf220943c4174e117d3b660859665baf1d43156230116185312d011. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [40]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def create_df(dataset):
    
    df = pd.DataFrame(dataset)
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    
    return df

In [28]:
datasets.keys()

dict_keys(['train', 'test', 'validation'])

In [60]:
datasets["train"][0]["goal"]

"When boiling butter, when it's ready, you can"

In [63]:
def preprocess_intents_json(segment):
    
    preprocessed_data = []
    
    for data in datasets[segment]:
        preprocessed_data.append(f"Goal: {data['goal']}\n")
        if data['label']=='1':
            preprocessed_data.append(f"Solution: {data['sol2']}\n")
        else:
            preprocessed_data.append(f"Solution: {data['sol1']}\n")
    
    return "".join(preprocessed_data)

def save_preprocessed_data(preprocessed_data, output_file):
    with open(output_file, "w") as f:
        f.write(preprocessed_data)


output_file = f"./data/data_train.txt"
preprocessed_data = preprocess_intents_json("train")
save_preprocessed_data(preprocessed_data, output_file)

output_file = f"./data/data_validation.txt"
preprocessed_data = preprocess_intents_json("validation")
save_preprocessed_data(preprocessed_data, output_file)

output_file = f"./data/data_test.txt"
preprocessed_data = preprocess_intents_json("test")
save_preprocessed_data(preprocessed_data, output_file)

## Preprocessing Data

In [70]:
import re
def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

text_data_train = read_txt("./data/data_train.txt")
text_data_train = re.sub(r'\n+', '\n', text_data_train).strip() 

text_data_test = read_txt("./data/data_validation.txt")
text_data_test = re.sub(r'\n+', '\n', text_data_test).strip() 

In [77]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [79]:
import transformers
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(text_data_train,text_data_test,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=text_data_train,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=text_data_test,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(text_data_train,text_data_test,tokenizer)

ImportError: 
TextDataset requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [ ]:
from transformers import GPT2Tokenizer, GPT2Model, Trainer, TrainingArguments
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

model = GPT2Model.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    prediction_loss_only=True,
)

In [ ]:
"C:\Users\shwet\Downloads\archive (2)\intents.json"